# hardware and OS
All steps are run on a ml.p3.2xlarge Amazon SageMaker Notebook instance.

Select kernel :conda_pytorch_p36. Jump directly to Install the requirements.

# Environment setup
before installing anything related to this project, first you need a version of PyTorch compatible with your GPU settings

with the 2 line script below you can check if your environment is ok.

In [ ]:
! wget --quiet "https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py"
! python collect_env.py

- first, check that your computer detect your graphic card ``GPU 0: ...``
- second, you need a driver to use your GPU for any application: ``Nvidia driver version: ...``
- third, you need some libraries to run any data science algorith on your GPU: ``cuDNN version: ...`` and ``CUDA runtime version: ...``
- fourth, you need to install Pytorch after having all the things above so that Pytorch is built using the CUDA library: ``PyTorch version: 1.5.0+cu101`` and ``CUDA used to build PyTorch: 10.1``
- Finally, if your are on windows, you might also need ``Microsoft Visual C++ 14.0``

If any of these are missing, the project setup will probably fail somewhere. Below are the instruction to fix your environment.

## Setup

In [ ]:
!gcc --version

## Driver



In [ ]:
# this command must work otherwise check the symLink /usr/local/cuda
!nvcc --version

## CUDA
Note that each version of CUDA has a minimum requirement concerning the version of the driver


cuda toolkit: https://developer.nvidia.com/cuda-10.1-download-archive-base?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1604&target_type=runfilelocal
example of thing you should install XD
```
wget https://developer.nvidia.com/compute/cuda/10.1/Prod/local_installers/cuda_10.1.105_418.39_linux.run
sh cuda_10.1.105_418.39_linux.run
```

## libnccl
Nvidia NCCL can be downloaded from: https://developer.nvidia.com/nccl/nccl-download (you need to create a free account)

## libcudnn
do not download ``libcudnn-dev``

then install it using this command ``dpkg -i libcudnn7_7.6.5.32-1+cuda10.1_amd64.deb``


## Pytorch
Get the command line that fits your hardware on this web site: https://pytorch.org/get-started/locally/
For example you should run something like the line below

In [ ]:
! pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# now run the command below to see if Pytorch detects your GPUs
import torch
torch.cuda.is_available()

# Install the requirements
[github source](https://github.com/KaihuaTang/Scene-Graph-Benchmark.pytorch)

## GCC
About 30 mins.

In [ ]:
%cd /usr/local/src/
!sudo wget http://ftp.gnu.org/gnu/gcc/gcc-7.3.0/gcc-7.3.0.tar.xz
!sudo tar xvf gcc-7.3.0.tar.xz
%cd gcc-7.3.0/
!sudo ./contrib/download_prerequisites
!sudo ./configure -enable-checking=release -enable-languages=c,c++ -disable-multilib
!sudo make -j4
!ls /usr/local/bin | grep gcc
!sudo make install
%cd ~
!find /usr/local/src/gcc-7.3.0/ -name "libstdc++.so*"
%cd /usr/lib64
!sudo cp /usr/local/src/gcc-7.3.0/stage1-x86_64-pc-linux-gnu/libstdc++-v3/src/.libs/libstdc++.so.6.0.24 .
!sudo mv libstdc++.so.6 libstdc++.so.6.old
!sudo ln -sv libstdc++.so.6.0.24 libstdc++.so.6
%cd /home/ec2-user/SageMaker/SGG

## Python libraries
Might take 5 to 10 mins to install all the rest. 

In [ ]:
! pip install ipython scipy h5py

In [ ]:
! pip install ninja yacs cython matplotlib tqdm opencv-python overrides

ends with `Successfully installed ninja-1.9.0.post1 overrides-3.0.0 yacs-0.1.7`

## install PyCOCO tools (cocoapi)

In [ ]:
! git clone https://github.com/cocodataset/cocoapi.git

In [ ]:
! cd cocoapi/PythonAPI; python setup.py build_ext install

ends with 

```
Finished processing dependencies for pycocotools==2.0
```



## install apex
it is a PyTorch extension for easy mixed precision and distributed training in Pytorch

In [ ]:
! git clone https://github.com/NVIDIA/apex.git
! cd apex ; python setup.py install

## install PyTorch Detection (Scene-Graph-Benchmark.pytorch)
We change the file name from `Scene-Graph-Benchmark.pytorch` to `Scene`
because *ninja* can not handel certain characters in the directory's name
([source](https://stackoverflow.com/questions/54569963/error-building-depfile-has-multiple-output-paths-ninja-build-stopped-subcomm ))

In [ ]:
!python setup.py build develop

ends with:


```
Installed /content/Scene
Processing dependencies for maskrcnn-benchmark==0.1
Finished processing dependencies for maskrcnn-benchmark==0.1
```
otherwise you can get:
~~~
RuntimeError: Error compiling objects for extension
~~~


The line above might not work for many reasons:
* ninja is not installed
* the folder name contains crasy characters like space, points
* other reasons

follow carefully the instructions above to avoid any problem :)

### Additionnal instructions for Windows:
You might incounter this issue https://github.com/facebookresearch/maskrcnn-benchmark/issues/547
then you might need to run that first:
```
set "VS150COMNTOOLS=C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Auxiliary\Build"
set CMAKE_GENERATOR=Visual Studio 16 2019 Win64
set DISTUTILS_USE_SDK=1
call "%VS150COMNTOOLS%\vcvarsall.bat" x64 -vcvars_ver=14.11
python setup.py build develop
call "C:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\vcvarsall.bat"  x64 -vcvars_ver=14.0
```

# DATASET
## VG images
Download the VG images part1 (9 Gb) part2 (5 Gb).

In [ ]:
! wget https://cs.stanford.edu/people/rak248/VG_100K_2/images.zip

In [ ]:
! wget https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip

Please Extract these images to the directory datasets/vg/VG_100K/ 

If you want to use other directory, please link it in `DATASETS['VG_stanford_filtered']['img_dir']` of `maskrcnn_benchmark/config/paths_catelog.py`

## SGG model

The following code does this automatically:

Download the [scene graphs](https://onedrive.live.com/embed?cid=22376FFAD72C4B64&resid=22376FFAD72C4B64%21779871&authkey=AA33n7BRpB1xa3I) to `Scene/datasets/vg/VG-SGG-with-attri.h5` (144 Mb)

or you can edit the path in `DATASETS['VG_stanford_filtered_with_attribute']['roidb_file']` of `maskrcnn_benchmark/config/paths_catelog.py`

In [ ]:
!pip install gdown

In [ ]:
# alternatively
! gdown "https://drive.google.com/uc?id=1h2XzeQgJNYgg3q66t1oujofbWvBIYMXG" -O ./maskrcnn_benchmark/data/datasets/vg/VG-SGG-with-attri.h5
# miror link for manual download https://onedrive.live.com/embed?cid=22376FFAD72C4B64&resid=22376FFAD72C4B64%21779871&authkey=AA33n7BRpB1xa3I

## Pretrained models

do not name any directory `checkpoints` with an ``s`` because you will not be able to explore it using jupyter notebook

In [ ]:
%%bash
mkdir ./checkpoint/pretrained_faster_rcnn/
cd ./checkpoint/pretrained_faster_rcnn/
gdown "https://drive.google.com/uc?id=1GoUdVlwZ8ekS7w_aWJ-tcXsx-ULCCjyI" -O log.txt
gdown "https://drive.google.com/uc?id=1Pj8gfFBouqaKzJVkOV6wsY8GU60z6Nrb" -O config.yml
gdown "https://drive.google.com/uc?id=1TRT3uX0tbqvIfNeL3bRGzVeqKS8SHtFa" -O model_final.pth
gdown "https://drive.google.com/uc?id=1Y1SnKGeQCBqGmIpUa8izy2EvUYLyPc89" -O VG_stanford_filtered_wth_attribute_train_statistics.cache
gdown "https://drive.google.com/uc?id=1_aRGThcciCvg0gFLr9EkhLIE92vEitfP" -O labels.json
gdown "https://drive.google.com/uc?id=1q6w_tZzhKTx70hgmQ-7Rnlt4EM60MmXp" -O last_checkpoint

If the links above are dead you can download and extract the files manually following this:

- [download the Faster R-CNN model](https://onedrive.live.com/embed?cid=22376FFAD72C4B64&resid=22376FFAD72C4B64%21779870&authkey=AH5CPVb9g5E67iQ),
- extract all the files to the directory `./checkpoints/pretrained_faster_rcnn`. 

To train your own Faster R-CNN model, please follow [the next section](https://github.com/KaihuaTang/Scene-Graph-Benchmark.pytorch#pretrained-models).


## SGDet Model

In [ ]:
!aws s3 cp s3://datalab2021/gree/causal_motif_sgdet.zip  ./checkpoints

In [ ]:
%cd checkpoints
!unzip causal_motif_sgdet.zip
%cd ..

# Setup done !
# Training and Testing
## Settings
The default settings are under

`configs/e2e_relation_X_101_32_8_FPN_1x.yaml` ([see on github](https://github.com/KaihuaTang/Scene-Graph-Benchmark.pytorch/blob/db02790a60bb9b9f7c270352820968b2f2089469/configs/e2e_relation_X_101_32_8_FPN_1x.yaml#L74))
and
`maskrcnn_benchmark/config/defaults.py` (todo find link in github)

The priority is in this order `command > yaml > defaults.py`

* For Predicate Classification (PredCls), we need to set:
```
MODEL.ROI_RELATION_HEAD.USE_GT_BOX True
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True
```

* For Unbiased-Causal-TDE Model:
```
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor
```

## Example 1 : (PreCls, Motif Model)
### Training Example 1 : (PreCls, Motif Model)

`CUDA_VISIBLE_DEVICES=0,1` <-- will use GPU 0 and 1

`python -m torch.distributed.launch` <-- will run the script across multiple GPUs

`--master_port 10025` <-- the value itself is not important, just use a free port

`--nproc_per_node=2` <-- [{num_gpus}](https://docs.fast.ai/distributed.html): this should correspond to the number of gpu you specified up


`tools/relation_train_net.py` <-- the script to run by the "torch.distributed". Mainly we want to train the model, or resume the training
`--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml"` <-- Config file for all what we didn't specify
`MODEL.ROI_RELATION_HEAD.USE_GT_BOX True ` <-- 
`MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True ` <-- This means that the ground truth object labels are provided as input to the model.

`MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor ` <-- round truth bounding boxes are provided as input to the model.

`SOLVER.IMS_PER_BATCH 12` <--        **this number must be divisible by the number of GPUs (2) used.**

`TEST.IMS_PER_BATCH 2` <--            **must be equal to the number of GPUs (2) used.**

`DTYPE "float16" ` <-- 

`SOLVER.MAX_ITER 50000 ` <-- number of epoch, there is also EarlyStopping implemented

`SOLVER.VAL_PERIOD 2000 ` <-- run validation every 2 000 epochs

`SOLVER.CHECKPOINT_PERIOD 2000 ` <-- create a checkpoint every 2000 epochs (1 hour or more)

`GLOVE_DIR /home/kaihua/glove ` <-- directory where the pretrained word embeddings will be downloaded and stored

`MODEL.PRETRAINED_DETECTOR_CKPT /home/kaihua/checkpoint/pretrained_faster_rcnn/model_final.pth ` <-- 

`OUTPUT_DIR /home/kaihua/checkpoints/motif-precls-exmp` <-- where the model is saved. if the directory is not empty then the training is automatically resumed. So you can for example stop training and add more GPUs



In [ ]:
!python \
tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX False \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
SOLVER.BASE_LR 0.08 \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE none SOLVER.IMS_PER_BATCH 12 TEST.IMS_PER_BATCH 1 DTYPE "float16" \
MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \
SOLVER.MAX_ITER 50000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 \
GLOVE_DIR ./glove \
MODEL.PRETRAINED_DETECTOR_CKPT ./checkpoints/pretrained_faster_rcnn/model_final.pth \
OUTPUT_DIR ./checkpoints/causal-motifs-sgdet-exmp

In [ ]:
!python \
tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX False \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE none SOLVER.IMS_PER_BATCH 8 TEST.IMS_PER_BATCH 2 DTYPE "float16" \
MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \
SOLVER.MAX_ITER 50000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 \
GLOVE_DIR ./glove \
MODEL.PRETRAINED_DETECTOR_CKPT ./checkpoints/pretrained_faster_rcnn/model_final.pth \
OUTPUT_DIR ./checkpoints/causal-motifs-sgdet-exmp

In [ ]:
CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port 10026 --nproc_per_node=2 tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" MODEL.ROI_RELATION_HEAD.USE_GT_BOX False MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE none MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs  SOLVER.IMS_PER_BATCH 12 TEST.IMS_PER_BATCH 2 DTYPE "float16" SOLVER.MAX_ITER 50000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 GLOVE_DIR ./glove MODEL.PRETRAINED_DETECTOR_CKPT /home/ec2-user/SageMaker/SGG/scene-graph-benchmark/pretrained_faster_rcnn/model_final.pth OUTPUT_DIR ./checkpoints/causal-motifs-sgdet-exmp
tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" MODEL.ROI_RELATION_HEAD.USE_GT_BOX True MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor SOLVER.IMS_PER_BATCH 12 TEST.IMS_PER_BATCH 2 DTYPE "float16" SOLVER.MAX_ITER 50000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 GLOVE_DIR ./glove MODEL.PRETRAINED_DETECTOR_CKPT ./checkpoints/pretrained_faster_rcnn/model_final.pth OUTPUT_DIR ./checkpoints/motif-precls-exmp

### Test

### Test Example 1 : (PreCls, Motif Model)
Better use only one GPU for testing
~~~
CUDA_VISIBLE_DEVICES=0
python -m torch.distributed.launch 
--master_port 10027 
--nproc_per_node=1
~~~

`tools/relation_test_net.py` <-- this line is the only that change

~~~
--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml"
MODEL.ROI_RELATION_HEAD.USE_GT_BOX True 
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True 
MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor 
~~~
`TEST.IMS_PER_BATCH 1` <----------------------------- must be equal to nproc_per_node
~~~
DTYPE "float16" 
GLOVE_DIR /home/kaihua/glove 
MODEL.PRETRAINED_DETECTOR_CKPT checkpoints/motif-precls-exmp 
OUTPUT_DIR checkpoints/motif-precls-exmp
~~~

In [ ]:
# evaluation
! cd "Scene/"; python -m torch.distributed.launch --master_port 10027 --nproc_per_node=8 tools/relation_test_net.py \
--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX True \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True \
MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor \
TEST.IMS_PER_BATCH 32 DTYPE "float16" \
GLOVE_DIR glove/ \
MODEL.PRETRAINED_DETECTOR_CKPT checkpoint/pretrained_faster_rcnn/model_final.pth \
OUTPUT_DIR checkpoint/motif-precls-exmp

## Train and test Example 2 : (SGCls, Causal, TDE, SUM Fusion, MOTIFS Model)

In [ ]:
!  python -m torch.distributed.launch --master_port 10026 --nproc_per_node=2 \
tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX True \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE none 
MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum \
MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \  
SOLVER.IMS_PER_BATCH 12 TEST.IMS_PER_BATCH 2 DTYPE "float16" \ 
SOLVER.MAX_ITER 50000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 \ 
GLOVE_DIR ./glove 
MODEL.PRETRAINED_DETECTOR_CKPT ./checkpoints/pretrained_faster_rcnn/model_final.pth 
OUTPUT_DIR ./checkpoints/causal-motifs-sgcls-exmp

In [ ]:
# evaluation
CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port 10028 --nproc_per_node=1 tools/relation_test_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" MODEL.ROI_RELATION_HEAD.USE_GT_BOX True MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs  TEST.IMS_PER_BATCH 1 DTYPE "float16" GLOVE_DIR /home/kaihua/glove MODEL.PRETRAINED_DETECTOR_CKPT /home/kaihua/checkpoints/causal-motifs-sgcls-exmp OUTPUT_DIR /home/kaihua/checkpoints/causal-motifs-sgcls-exmp

## Train and test Example 3 : (SGDet, Causal, TDE, SUM Fusion, MOTIFS Model)

In [ ]:
!python \
tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX False \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE none SOLVER.IMS_PER_BATCH 8 TEST.IMS_PER_BATCH 1 DTYPE "float16" \
MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \
SOLVER.MAX_ITER 50000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 \
GLOVE_DIR ./glove \
MODEL.PRETRAINED_DETECTOR_CKPT ./checkpoints/pretrained_faster_rcnn/model_final.pth \
OUTPUT_DIR ./checkpoints/causal-motifs-sgdet02-exmp

In [ ]:
CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port 10026 --nproc_per_node=1 \
tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX False \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR VCTreePredictor \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE none MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum \
MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER vctree SOLVER.IMS_PER_BATCH 12 TEST.IMS_PER_BATCH 1 \
DTYPE "float16" SOLVER.MAX_ITER 50000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 \
GLOVE_DIR ./glove \
MODEL.PRETRAINED_DETECTOR_CKPT ./checkpoints/pretrained_faster_rcnn/model_final.pth \
OUTPUT_DIR ./checkpoints/causal-vctree-sgdet03-exmp

## Test

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port 10055 --nproc_per_node=1 ./tools/relation_test_net.py --config-file "./configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX False MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \
TEST.IMS_PER_BATCH 1 DTYPE "float16" GLOVE_DIR ./glove MODEL.PRETRAINED_DETECTOR_CKPT ./checkpoints/causal_motif_sgdet OUTPUT_DIR ./checkpoints/causal_motif_sgdet TEST.CUSTUM_EVAL True \
TEST.CUSTUM_PATH ./custom_images DETECTED_SGG_DIR ./sgdet_output